In [1]:
import pandas as pd
from keplergl import KeplerGl

In [3]:
# Updated script to generate heatmap based on region and order frequency

import pandas as pd
from keplergl import KeplerGl

# File paths
orders_file_path = 'Global Superstore Orders 2016.xlsx'
uscities_file_path = 'uscities.csv'

# Load the orders data with the Region column
orders_data = pd.read_excel(
    orders_file_path,
    sheet_name='Orders',
    usecols=['Region', 'State', 'City', 'Country']
)

# Prompt to filter by region (example region: 'West')
selected_region = "Eastern US"  # You can adjust this value or make it dynamic

# Filter for the selected region and ensure it's in the United States
region_orders = orders_data[(orders_data['Region'] == selected_region) & (orders_data['Country'] == 'United States')]

# Identify unique states in the selected region
region_states = region_orders['State'].unique()

# Load the cities data for latitude, longitude, and state
uscities_df = pd.read_csv(uscities_file_path)

# Filter for cities in the states of the selected region
region_cities = uscities_df[uscities_df['state_name'].isin(region_states)][['city', 'lat', 'lng', 'state_name']]

# Merge region orders with the geospatial data of cities in the region
region_geospatial = region_orders.merge(
    region_cities,
    left_on=['City', 'State'],
    right_on=['city', 'state_name'],
    how='inner'  # Ensure only matched cities are included
)

# Group by city and calculate the frequency of orders
region_geospatial = region_geospatial.groupby(['City', 'lat', 'lng'], as_index=False).size()

# Rename the 'size' column to 'OrderFrequency' for clarity
region_geospatial.rename(columns={'size': 'OrderFrequency'}, inplace=True)

# Prepare Kepler.gl visualization
kepler_map = KeplerGl(height=700)
kepler_map.add_data(data=region_geospatial, name=f'{selected_region} Region Orders Heatmap')

# Save the map configuration and data for Kepler.gl
region_data_path = f'{selected_region.lower()}_orders_frequency.csv'
kepler_output_path = f'{selected_region.lower()}_orders_frequency_heatmap.html'

# Save the region geospatial data with order frequency
region_geospatial.to_csv(region_data_path, index=False)
print(f"Region geospatial data with order frequency saved to {region_data_path}")


kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Region geospatial data with order frequency saved to eastern us_orders_frequency.csv


KeplerGl(data={'Eastern US Region Orders Heatmap': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [4]:
# Save the Kepler map as an HTML file
kepler_map.save_to_html(file_name=kepler_output_path)
print(f"Kepler map saved as {kepler_output_path}")

Map saved to eastern us_orders_frequency_heatmap.html!
Kepler map saved as eastern us_orders_frequency_heatmap.html
